In [91]:
import pandas as pd
import numpy as np
import json
from IPython.display import display

## sklearn imports
import sklearn.linear_model
import sklearn.cross_validation
import sklearn.tree
import sklearn.ensemble
import sklearn.preprocessing
import sklearn.feature_selection
import sklearn.pipeline
import sklearn.grid_search

# Data Science Analysis

### 1. Descriptive
* Using pandas and numpy, preprocess the data by:
    - reading in the data
    - transforming features and labels to numerical data
    Use OneHotEncoder object in scikit-learn, or get_dummies in pandas
    - pick a modeling strategy (classification vs. regression)
    - make a train/test split using cross_val_score
    - evaluate several different models

### 2. Exploratory

### 3. Inferential

### 4. Prediction

### 5. Mechanistic

#### Some References:
1. https://civisanalytics.com/blog/data-science/2016/01/06/workflows-python-using-pipeline-gridsearchcv-for-compact-code/
2. http://blog.datadive.net/selecting-good-features-part-iii-random-forests/

## Use Pandas and Numpy For Preprocessing

In [33]:
features_df = pd.DataFrame.from_csv("well_train_values.csv")
labels_df   = pd.DataFrame.from_csv("well_train_labels.csv") 
display(labels_df.head(10))
display(features_df.head())

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional
9944,functional
19816,non functional
54551,non functional
53934,non functional


,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [34]:
def label_map(y):
    if y=="functional":
        return 2
    elif y=="functional needs repair":
        return 1
    else:
        return 0
labels_df = labels_df.applymap(label_map)
display(labels_df.head())

,status_group
id,
69572,2
8776,2
34310,2
67743,0
19728,2


In [43]:
print 'The columns that are categorical are: '
for column in features_df.columns:
    col_df = features_df[column]
    if col_df.dtype == 'object':
        print '\t', column

The columns that are categorical are: 
	date_recorded
	funder
	installer
	wpt_name
	basin
	subvillage
	region
	lga
	ward
	public_meeting
	recorded_by
	scheme_management
	scheme_name
	permit
	extraction_type
	extraction_type_group
	extraction_type_class
	management
	management_group
	payment
	payment_type
	water_quality
	quality_group
	quantity
	quantity_group
	source
	source_type
	source_class
	waterpoint_type
	waterpoint_type_group


In [47]:
def transform_feature( df, column_name ):
    unique_values = set( df[column_name].tolist() )
    transformer_dict = {}
    
    # enumerate each unique category into a list of values
    for ii, value in enumerate(unique_values):
        transformer_dict[value] = ii

    def label_map(y):
        return transformer_dict[y]
    
    df[column_name] = df[column_name].apply( label_map )
    return df

### list of column names indicating which columns to transform; 
### this is just a start!  Use some of the print( labels_df.head() )
### output upstream to help you decide which columns get the
### transformation
names_of_columns_to_transform = ["funder", "installer", "wpt_name", "basin", "subvillage",
                    "region", "lga", "ward", "public_meeting", "recorded_by",
                    "scheme_management", "scheme_name", "permit",
                    "extraction_type", "extraction_type_group",
                    "extraction_type_class",
                    "management", "management_group",
                    "payment", "payment_type",
                    "water_quality", "quality_group", "quantity", "quantity_group",
                    "source", "source_type", "source_class",
                    "waterpoint_type", "waterpoint_type_group"]

for column in names_of_columns_to_transform:
    features_df = transform_feature( features_df, column )
    
### remove the "date_recorded" column--we're not going to make use
### of time-series data today
if 'date_recorded' in features_df.columns:
    features_df.drop("date_recorded", axis=1, inplace=True)

# display the new head and the values of the columns
display( features_df.head() )
display(features_df.columns.values)

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,1539,1390,1749,34.938093,-9.856322,15203,0,5,9384,...,4,0,0,2,2,1,1,1,1,1
8776,0.0,774,1399,136,34.698766,-2.147466,5611,0,7,3466,...,5,0,0,1,1,6,4,2,1,1
34310,25.0,906,686,1400,37.460664,-3.821329,6138,0,6,11348,...,1,0,0,2,2,7,5,2,4,1
67743,0.0,590,263,556,38.486161,-11.155298,1252,0,1,8105,...,5,0,0,0,0,2,6,1,4,1
19728,0.0,1341,0,537,31.130847,-1.825359,29682,0,7,18660,...,5,0,0,3,3,6,4,2,1,1


array(['amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'wpt_name', 'num_private', 'basin', 'subvillage',
       'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source',
       'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'], dtype=object)

In [48]:
X = features_df.as_matrix()
y = labels_df["status_group"].tolist()

## 01: Logistic Regression, Decision Tree and Random Forest

In [63]:
N = float(len(y))
for i in np.unique(y):
    print 'Baseline for: ', i, ' ', y.count(i), ' ', y.count(i)/N   

Baseline for:  0   22824   0.384242424242
Baseline for:  1   4317   0.0726767676768
Baseline for:  2   32259   0.543080808081


In [51]:
# regular ridge regularization
clf = sklearn.linear_model.LogisticRegression()
score = sklearn.cross_validation.cross_val_score(clf, X, y)
print score

[ 0.6879798   0.68373737  0.68469697]


In [53]:
# lasso regularization
clf = sklearn.linear_model.LogisticRegression(penalty='l1')
score = sklearn.cross_validation.cross_val_score(clf, X, y)
print score

[ 0.68893939  0.68631313  0.68691919]


In [62]:
# decision tree classifier
clf = sklearn.tree.DecisionTreeClassifier()
score = sklearn.cross_validation.cross_val_score( clf, X, y )
print( score )

# random forest classifier
clf = sklearn.ensemble.RandomForestClassifier()
score = sklearn.cross_validation.cross_val_score( clf, X, y )
print( score )

[ 0.73893939  0.73858586  0.73146465]
[ 0.7880303   0.78727273  0.78373737]


## 02: Further PreProcessing


In [65]:
def hot_encoder(df, column_name):
    column = df[column_name].tolist()
    column = np.reshape( column, (len(column), 1) )  ### needs to be an N x 1 numpy array
    enc = sklearn.preprocessing.OneHotEncoder()
    enc.fit( column )
    new_column = enc.transform( column ).toarray()

    ### making titles for the new columns, and appending them to dataframe
    for ii in range( len(new_column[0]) ):
        this_column_name = column_name+"_"+str(ii)
        df[this_column_name] = new_column[:,ii]
    return df


In [70]:
print(features_df.columns.values)

try:
    features_df.drop( "funder", axis=1, inplace=True )
    features_df.drop( "installer", axis=1, inplace=True )
    features_df.drop( "wpt_name", axis=1, inplace=True )
    features_df.drop( "subvillage", axis=1, inplace=True )
    features_df.drop( "ward", axis=1, inplace=True )

    names_of_columns_to_transform.remove("funder")
    names_of_columns_to_transform.remove("installer")
    names_of_columns_to_transform.remove("wpt_name")
    names_of_columns_to_transform.remove("subvillage")
    names_of_columns_to_transform.remove("ward")
    
    for feature in names_of_columns_to_transform:
        features_df = hot_encoder( features_df, feature )
except:
    print 'Already removed large columns'



['amount_tsh' 'gps_height' 'longitude' ..., 'waterpoint_type_group_3'
 'waterpoint_type_group_4' 'waterpoint_type_group_5']
Already removed large columns


In [71]:
display(features_df.head())

,amount_tsh,gps_height,longitude,latitude,num_private,basin,region,region_code,district_code,lga,...,waterpoint_type_3,waterpoint_type_4,waterpoint_type_5,waterpoint_type_6,waterpoint_type_group_0,waterpoint_type_group_1,waterpoint_type_group_2,waterpoint_type_group_3,waterpoint_type_group_4,waterpoint_type_group_5
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,1390,34.938093,-9.856322,0,5,7,11,5,65,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8776,0.0,1399,34.698766,-2.147466,0,7,3,20,2,86,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34310,25.0,686,37.460664,-3.821329,0,6,4,21,4,11,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
67743,0.0,263,38.486161,-11.155298,0,1,20,90,63,122,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19728,0.0,0,31.130847,-1.825359,0,7,1,18,1,70,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [81]:
X = features_df.as_matrix()
y = labels_df["status_group"].as_matrix()
y = np.asarray(y)
print X.shape
print y.shape
selecter = sklearn.feature_selection.SelectKBest(k=100)
selected_x = selecter.fit_transform(X, y)

print selected_x.shape

(59400, 3031)
(59400,)


/usr/local/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [ 12 191] are constant.
  UserWarning)


(59400, 100)


In [82]:
## show new classification results from the new features

# regular ridge regularization
clf = sklearn.linear_model.LogisticRegression()
score = sklearn.cross_validation.cross_val_score(clf, X, y)
print score

# lasso regularization
clf = sklearn.linear_model.LogisticRegression(penalty='l1')
score = sklearn.cross_validation.cross_val_score(clf, X, y)
print score

# decision tree classifier
clf = sklearn.tree.DecisionTreeClassifier()
score = sklearn.cross_validation.cross_val_score( clf, X, y )
print( score )

# random forest classifier
clf = sklearn.ensemble.RandomForestClassifier()
score = sklearn.cross_validation.cross_val_score( clf, X, y )
print( score )

[ 0.74813131  0.75409091  0.71974747]
[ 0.76893939  0.76727273  0.76823232]
[ 0.75570707  0.75767677  0.75484848]
[ 0.77909091  0.78        0.77944444]


## 03: Convert Process Into Sklearn Pipeline

In [89]:
select = sklearn.feature_selection.SelectKBest(k=100)
clf = sklearn.ensemble.RandomForestClassifier()

# define steps for pipeline
steps = [('feature_selection', select),
         ('random_forest', clf)]
pipeline = sklearn.pipeline.Pipeline(steps)

print pipeline

X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(X, y, test_size=0.3, random_state=42)

## fit, predict, test pipeline
pipeline.fit(X_train, y_train)
y_predict = pipeline.predict(X_test)
report = sklearn.metrics.classification_report(y_test, y_predict)

print report

Pipeline(steps=[('feature_selection', SelectKBest(k=100, score_func=<function f_classif at 0x10d1f5050>)), ('random_forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
             precision    recall  f1-score   support

          0       0.77      0.77      0.77      6803
          1       0.41      0.37      0.39      1293
          2       0.81      0.81      0.81      9724

avg / total       0.76      0.76      0.76     17820



In [90]:
score = sklearn.cross_validation.cross_val_score(clf, X, y )
print( score )

[ 0.78        0.78252525  0.78080808]


## 04: Add Grid Search On CLF Parameters
For this example, it will be random forest

In [94]:
params = dict(feature_selection__k=[100, 200],
             random_forest__n_estimators=[50, 100, 200],
             random_forest__min_samples_split=[2, 3, 4, 5, 10])

print json.dumps(params, indent=4)

{
    "random_forest__n_estimators": [
        50, 
        100, 
        200
    ], 
    "feature_selection__k": [
        100, 
        200
    ], 
    "random_forest__min_samples_split": [
        2, 
        3, 
        4, 
        5, 
        10
    ]
}


In [96]:
cv = sklearn.grid_search.GridSearchCV(pipeline, param_grid=params)
cv.fit(X_train, y_train)
y_predict = cv.predict(X_test)
report = sklearn.metrics.classification_report(y_test, y_predict)

/usr/local/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [  12  191  193  206  211  229  232  238  256  269  270  273  277  278  287
  290  291  302  303  307  310  313  315  322  324  325  335  344  347  353
  359  364  371  378  383  392  403  410  419  422  423  429  432  433  437
  440  444  450  453  459  462  474  479  481  485  490  496  502  504  507
  509  521  523  529  533  535  537  538  539  540  541  542  546  547  549
  551  558  561  566  570  583  585  593  597  602  609  620  621  639  647
  648  649  656  677  683  684  691  693  700  705  727  734  735  738  748
  749  751  755  761  767  777  781  782  793  795  797  801  810  812  816
  822  825  826  829  836  845  847  853  855  856  862  863  867  870  873
  889  890  894  903  905  910  912  944  946  947  950  961  965  975  977
  990  994 1004 1005 1011 1012 1024 1026 1031 1034 1036 1044 1048 1051 1052
 1053 1062 1070 1077 1091 1096 1097 1099 1100 11

In [101]:
print report
names = features_df.columns.tolist()
print sorted(zip(map(lambda x: round(x, 4), clf.feature_importances_), names),reverse=True)

             precision    recall  f1-score   support

          0       0.83      0.77      0.80      6803
          1       0.57      0.30      0.39      1293
          2       0.80      0.89      0.84      9724

avg / total       0.79      0.80      0.79     17820

[(0.2726, 'gps_height'), (0.1348, 'amount_tsh'), (0.0397, 'recorded_by'), (0.0397, 'lga_14'), (0.0375, 'num_private'), (0.0274, 'latitude'), (0.0258, 'lga_11'), (0.0231, 'longitude'), (0.0215, 'public_meeting'), (0.0201, 'construction_year'), (0.0153, 'region'), (0.0153, 'basin'), (0.0145, 'lga_36'), (0.0143, 'region_code'), (0.0141, 'region_9'), (0.0134, 'region_12'), (0.0108, 'region_5'), (0.0108, 'extraction_type'), (0.0101, 'lga_12'), (0.0097, 'basin_8'), (0.0092, 'basin_3'), (0.0077, 'region_16'), (0.0076, 'lga_15'), (0.0071, 'district_code'), (0.0069, 'scheme_management'), (0.0068, 'region_18'), (0.0067, 'lga'), (0.006, 'scheme_name'), (0.006, 'basin_4'), (0.0056, 'lga_31'), (0.0052, 'lga_5'), (0.0049, 'region_7'), (